## 라마팩토리 설치

In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 22679, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 22679 (delta 14), reused 4 (delta 4), pack-reused 22652 (from 4)
Receiving objects: 100% (22679/22679), 44.23 MiB | 33.32 MiB/s, done.
Resolving deltas: 100% (16486/16486), done.


In [28]:
cd LLaMA-Factory

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
!pip install e ".[torch,metrics]"

Processing /workspace/LLaMA-Factory
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 165.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 103.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
!pip3 install deepspeed datasets vllm scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


## GPU 사용 유무
- 사용 불가일 경우 에러 메시지 발생

In [31]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## 데이터셋 로드

In [32]:
import json
import os
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import IPython.display as display

허깅페이스에 이미 올려놓은 데이터를 datasets 패키지를 사용하여 다운로드하고 첫번째 데이터를 출력합니다.

In [33]:
# 데이터셋 로드
print("데이터셋 다운로드 중...")
dataset = load_dataset("iamjoon/finance_news_summarizer")
train_data = dataset["train"]

# 샘플 데이터 확인
print("\n첫 번째 샘플:")
display.display(train_data[0])

데이터셋 다운로드 중...

첫 번째 샘플:


{'system_prompt': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들

## 학습 데이터와 테스트 데이터의 분리

학습 데이터와 테스트 데이터를 8:2로 분할합니다.

In [34]:
# Dataset을 리스트로 변환
train_data_list = list(train_data)

In [35]:
# 학습/테스트 데이터 분할 (8:2)
train_split, test_split = train_test_split(train_data_list, test_size=0.2, random_state=42)

## 라마팩토리 특유의 형식으로 변환

라마팩토리에서는 아래의 형식으로 데이터를 저장해야 합니다.
- 'system': 시스템 프롬프트
- 'instruction': 유저 프롬프트
- 'output': 레이블

'input'은 크게 신경쓰지 않으셔도 됩니다. 그냥 빈 값으로 두어도 무방합니다.

In [36]:
# LLaMA-Factory 형식으로 변환하는 함수
def convert_to_llama_factory_format(data_split):
    converted = []
    for item in data_split:
        if isinstance(item["assistant"], dict):
            assistant_text = json.dumps(item["assistant"], ensure_ascii=False)
        else:
            assistant_text = item["assistant"]

        preprocessed_item = {
            "instruction": item["user_prompt"],
            "input": "",
            "output": assistant_text,
            "system": item["system_prompt"]
        }
        converted.append(preprocessed_item)
    return converted

In [37]:
# 변환
print("\n학습/테스트 데이터셋 변환 중...")
llama_train = convert_to_llama_factory_format(train_split)
llama_test = convert_to_llama_factory_format(test_split)


학습/테스트 데이터셋 변환 중...


In [39]:
# 샘플 데이터 확인
print("\n첫 번째 샘플:")
display.display(llama_train[0])


첫 번째 샘플:


{'instruction': '지난해 국민연금 수익률 10.86%‥기금운용본부 성과급 기본급의 68%\n지난해 국민연금기금의 금융부문 운용 수익률이 10.86%로 확정되면서 올해 기금운용본부의 성과급 지급률이 기본급 대비 67.7%로 정해졌다. 국민연금기금운용위원회는 1일 서울 중구 프레지던트호텔에서 제4차 회의를 열고 2021년도 국민연금 기금운용 성과평가 와 2021년도 국민연금 기금운용본부 성과급 지급률 을 심의·의결했다. 이에 따르면 지난해 국민연금의 기금운용 수익률은 전년 9.58% 보다 소폭 상승한 10.86%로 확정됐다. 이는 지난 5월 기획재정부가 국무회의에 보고한 기금평가 결과 수치와 동일하다. 국민연금 내 기금운용본부가 설립된 이후 두 자릿수 수익률을 기록한 것은 2009년 10.8% 2010년 10.6% 2019년 11.34% 이후 이번이 네 번째다. 위원회에 따르면 지난해 자산군별 수익률은 해외주식 29.77% 대체투자 23.97% 국내주식 5.88% 해외채권 7.26% 국내채권 1.25% 등으로 대체투자와 해외주식 수익률 상승이 전체 기금 수익률을 견인했다. 성과급 지급률 67.7%는 전년 86.7% 대비 19%포인트 하락했는데 이는 기준수익률 대비 초과성과가 0.99%포인트에서 0.04%포인트로 감소했기 때문이다. 성과급 지급률은 최근 3년 성과를 기준으로 각 연도 성과를 5 3 2 비율로 반영한다. 한편 위원회는 이날 국민연금성과 제고를 위한 2022년 정책제언 과제를 확정했다. 여기에는 수익 원천 다변화를 위한 자산배분 유연성 강화 체계 마련 위험관리체계 운영의 점검 및 개선방안 검토 대체투자 성과평가 벤치마크의 타당성 및 정합성 검토 등의 내용이 포함됐다.',
 'input': '',
 'output': '{"is_stock_related": false, "negative_impact_stocks": null, "negative_keywords": null, "positive_impact_stocks": null, "positiv

In [40]:
print('시스템 프롬프트')
print('==' * 50)
print(llama_train[0]['system'])

시스템 프롬프트
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들을 파이썬 문자열 리스트 형태로 작성하십시오. 기술명,

In [41]:
print('유저 프롬프트')
print('==' * 50)
print(llama_train[0]['instruction'])

유저 프롬프트
지난해 국민연금 수익률 10.86%‥기금운용본부 성과급 기본급의 68%
지난해 국민연금기금의 금융부문 운용 수익률이 10.86%로 확정되면서 올해 기금운용본부의 성과급 지급률이 기본급 대비 67.7%로 정해졌다. 국민연금기금운용위원회는 1일 서울 중구 프레지던트호텔에서 제4차 회의를 열고 2021년도 국민연금 기금운용 성과평가 와 2021년도 국민연금 기금운용본부 성과급 지급률 을 심의·의결했다. 이에 따르면 지난해 국민연금의 기금운용 수익률은 전년 9.58% 보다 소폭 상승한 10.86%로 확정됐다. 이는 지난 5월 기획재정부가 국무회의에 보고한 기금평가 결과 수치와 동일하다. 국민연금 내 기금운용본부가 설립된 이후 두 자릿수 수익률을 기록한 것은 2009년 10.8% 2010년 10.6% 2019년 11.34% 이후 이번이 네 번째다. 위원회에 따르면 지난해 자산군별 수익률은 해외주식 29.77% 대체투자 23.97% 국내주식 5.88% 해외채권 7.26% 국내채권 1.25% 등으로 대체투자와 해외주식 수익률 상승이 전체 기금 수익률을 견인했다. 성과급 지급률 67.7%는 전년 86.7% 대비 19%포인트 하락했는데 이는 기준수익률 대비 초과성과가 0.99%포인트에서 0.04%포인트로 감소했기 때문이다. 성과급 지급률은 최근 3년 성과를 기준으로 각 연도 성과를 5 3 2 비율로 반영한다. 한편 위원회는 이날 국민연금성과 제고를 위한 2022년 정책제언 과제를 확정했다. 여기에는 수익 원천 다변화를 위한 자산배분 유연성 강화 체계 마련 위험관리체계 운영의 점검 및 개선방안 검토 대체투자 성과평가 벤치마크의 타당성 및 정합성 검토 등의 내용이 포함됐다.


In [42]:
print('어시스턴트 답변')
print('==' * 50)
print(llama_train[0]['output'])

어시스턴트 답변
{"is_stock_related": false, "negative_impact_stocks": null, "negative_keywords": null, "positive_impact_stocks": null, "positive_keywords": null, "reason_for_negative_impact": null, "reason_for_positive_impact": null, "summary": "지난해 국민연금 기금운용 수익률이 10.86%로 확정되면서, 기금운용본부의 성과급 지급률이 기본급 대비 67.7%로 결정되었다. 주요 수익률은 해외주식과 대체투자에서 높았으며, 국민연금 내 기금운용본부가 설립된 이래 네 번째 두 자릿수 수익률을 기록했다. 성과급 지급률은 기준수익률 대비 초과성과가 감소하여 전년 대비 19%포인트 하락했다. 또한, 국민연금 성과 제고를 위한 2022년 정책제언 과제가 확정되었다."}


In [44]:
# output 부분은 반드시 문자열이어야 합니다.
type(llama_train[0]['output'])

str

라마팩토리에서는 data 디렉토리 안에 json 파일로 파일을 저장해야 합니다.

In [38]:
# 저장
print("\nJSON 파일로 저장 중...")
os.makedirs("data", exist_ok=True)

with open("data/finance_news_summarizer_train.json", "w", encoding="utf-8") as f:
    json.dump(llama_train, f, ensure_ascii=False, indent=2)

with open("data/finance_news_summarizer_test.json", "w", encoding="utf-8") as f:
    json.dump(llama_test, f, ensure_ascii=False, indent=2)

print("완료되었습니다.")


JSON 파일로 저장 중...
완료되었습니다.


## dataset_info.json 업데이트

라마팩토리에서는 data 디렉토리 내에 데이터를 저장하고나서 반드시 dataset_info.json에 데이터를 등록해야 합니다.

In [45]:
# 데이터셋 정보 파일 업데이트
dataset_info_path = "data/dataset_info.json"

if os.path.exists(dataset_info_path):
    with open(dataset_info_path, "r", encoding="utf-8") as f:
        dataset_info = json.load(f)
else:
    dataset_info = {}

# finance_news_summarizer 데이터셋 정보 추가
dataset_info["finance_news_summarizer"] = {
    "file_name": "finance_news_summarizer_train.json",
    "columns": {
        "prompt": "instruction", # 유저 프롬프트
        "query": "input", # 실제 사용되지는 않지만 형식적으로 넣어준다.
        "response": "output", # 어시스턴트 답변
        "system": "system" # 시스템 프롬프트
    }
}

In [46]:
# 업데이트된 데이터셋 정보 저장
with open(dataset_info_path, "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, ensure_ascii=False, indent=2)

print(f"데이터셋 정보가 {dataset_info_path}에 업데이트되었습니다.")

데이터셋 정보가 data/dataset_info.json에 업데이트되었습니다.


## 학습 파라미터 설정

라마팩토리에서는 학습 파라미터를 설정하고나서 설정값을 json 파일로 저장해야 합니다.

In [47]:
# 학습 구성 생성
args = dict(
    stage="sft",                                               # 지도 학습 미세 조정 수행
    do_train=True,
    model_name_or_path="NCSOFT/Llama-VARCO-8B-Instruct",       # 한글 사용 가능한 라마 모델
    dataset="finance_news_summarizer",                         # 변환한 데이터셋 사용
    template="llama3",                                         # llama3 프롬프트 템플릿 사용
    finetuning_type="lora",                                    # 메모리 절약을 위해 LoRA 어댑터 사용
    lora_target="all",                                         # 모든 선형 레이어에 LoRA 어댑터 연결
    output_dir="llama3_finance_lora",                          # LoRA 어댑터 저장 경로
    per_device_train_batch_size=2,                             # 마이크로 배치 크기
    gradient_accumulation_steps=4,                             # 그래디언트 누적 단계
    lr_scheduler_type="cosine",                                # 코사인 학습률 스케줄러 사용
    learning_rate=1e-5,                                        # 학습률
    num_train_epochs=3.0,                                      # 학습 에포크 수
    bf16=True,                                                 # bfloat16 혼합 정밀도 학습 사용
    report_to="none",                                          # wandb 로깅 비활성화

    # 멀티 GPU 및 DeepSpeed 설정
    deepspeed="examples/deepspeed/ds_z3_config.json",          # DeepSpeed ZeRO-3 설정 파일 경로
    ddp_find_unused_parameters=False,                          # 멀티 GPU 학습을 위한 설정
)

In [48]:
# 학습 구성 저장
json.dump(args, open("/workspace/LLaMA-Factory/train_finance.json", "w", encoding="utf-8"), indent=2)

print("학습 구성이 저장되었습니다. 이제 학습을 시작합니다.")

학습 구성이 저장되었습니다. 이제 학습을 시작합니다.


원래는 아래의 명령어로 학습을 시작하면 되지만 로그가 너무 많이 출력되어서 스크롤이 너무 길게 나온다는 단점이 있습니다.

In [49]:
# 2개 GPU로 학습 실행
# !CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli train train_finance.json

# 위 명령어를 그대로 실행하면 주피터 노트북에서 로그가 출력되서 보기가 너무 힘들다.
# 따라서 아래의 명령어를 사용하면 log 파일로 로그를 관리하면서 볼 수 있다.

따라서 위 명령어를 파이썬 코드로 변경하고 로그는 train.log에 실시간으로 저장되도록 합니다.

In [50]:
import subprocess

# 직접 학습 명령어를 실행하고 로그 파일로 출력 리디렉션
with open("train.log", "w") as log_file:
    process = subprocess.Popen(
        ["llamafactory-cli", "train", "train_finance.json"],
        env=dict(os.environ, CUDA_VISIBLE_DEVICES="0,1"),
        stdout=log_file,
        stderr=subprocess.STDOUT
    )

print(f"학습이 백그라운드에서 시작되었습니다. 프로세스 ID: {process.pid}")
print("로그는 /workspace/LLaMA-Factory/train.log에 저장됩니다.")

학습이 백그라운드에서 시작되었습니다. 프로세스 ID: 1968
로그는 /workspace/LLaMA-Factory/train.log에 저장됩니다.


In [56]:
# 로그 파일의 최근 내용 확인
!tail -n 50 /workspace/LLaMA-Factory/train.log

[INFO|trainer.py:2681] 2025-04-08 04:57:32,526 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 868.9646, 'train_samples_per_second': 2.734, 'train_steps_per_second': 0.169, 'train_loss': 0.7034192766462054, 'epoch': 2.95}
100%|██████████| 147/147 [14:28<00:00,  5.91s/it]
[INFO|trainer.py:3984] 2025-04-08 04:57:41,602 >> Saving model checkpoint to llama3_finance_lora
[INFO|configuration_utils.py:693] 2025-04-08 04:57:42,021 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NCSOFT--Llama-VARCO-8B-Instruct/snapshots/fe2d9358a2d35451c04e4589b47e361cfacd350d/config.json
[INFO|configuration_utils.py:765] 2025-04-08 04:57:42,023 >> Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_r